# 推荐系统 on MovieLens的数据集

# 用户和Movis关联关系处理（训练数据）

In [1]:
#保存数据
import cPickle

import itertools

#处理事件字符串
import datetime

import numpy as np
import scipy.io as sio
import scipy.sparse as ss

#用户数目和电影数目
from utils import n_Users, n_Movies

from collections import defaultdict

In [12]:
 """
u1.base/u1.test 有4列：
user：用户ID
movieid：movieID
rating：用户打分
timestamp：表示用户打分的时间，unix seconds since 1/1/1970 UTC
"""

#统计每个用户参加的事件   / 每个事件参加的用户
moviesForUser = defaultdict(set)
usersForMovie = defaultdict(set)
    
for filename in ["u1.base", "u1.test"]:
    #user id | item id | rating | timestamp
    for line in open(filename,'r'):  #对每条记录
        (user,movieid,rating,ts)=line.split('\t')
        
        user_int = int(user)-1   #索引从0开始
        movieid_int = int(movieid)-1
               
        #倒排表
        moviesForUser[user_int].add(movieid_int)    #该用户对这个电影进行了打分
        usersForMovie[movieid_int].add(user_int)    #该事件被用户参加

##统计每个用户参加的事件
cPickle.dump(moviesForUser, open("PE_moviesForUser.pkl", 'wb'))
##统计参加事件的用户
cPickle.dump(usersForMovie, open("PE_usersForMoive.pkl", 'wb'))

#用户-电影关系矩阵表（打分），可用于后续LFM/SVD++处理的输入
#这是一个稀疏矩阵，记录用户对每个电影的打分
userMovieScores = ss.dok_matrix((n_Users, n_Movies))
    
#从训练文件读取数据
filename = "u1.base"
#user id | item id | rating | timestamp
for line in open(filename,'r'):  #对每条记录
    (user,movieid,rating,ts)=line.split('\t')
    
    #下标从0开始
    user_int = int(user)-1   #索引从0开始
    movieid_int = int(movieid)-1

    userMovieScores[user_int, movieid_int] = float(rating)
 
#保存用户-事件关系矩阵R，以备后用
sio.mmwrite("PE_userMovieScores", userMovieScores)


# 为了防止不必要的计算，我们找出来所有关联的用户 或者 关联的event
# 所谓的关联用户，指的是至少在同一个event上有行为的用户pair
# 关联的event指的是至少同一个user有行为的event pair
uniqueUserPairs = set()
uniqueMoviePairs = set()
for movie in range(n_Movies):
    users = usersForMovie[movie]
    if len(users) > 2:
        uniqueUserPairs.update(itertools.combinations(users, 2))
        
for user in range(n_Users):
    movies = moviesForUser[user]
    if len(movies) > 2:
        uniqueMoviePairs.update(itertools.combinations(movies, 2))
 
#保存用户-事件关系对索引表
cPickle.dump(uniqueUserPairs, open("FE_uniqueUserPairs.pkl", 'wb'))
cPickle.dump(uniqueMoviePairs, open("PE_uniqueMoviePairs.pkl", 'wb'))

print "finished reading train & test"

finished reading train & test
